## 프로젝트 - Movielens 영화 추천 실습

In [1]:
import numpy as np
import scipy
import implicit

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


## 데이터 준비와 전처리

In [2]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


영화 소문자 변환

In [6]:
movies['title'] = movies['title'].str.lower() # 검색을 쉽게 하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
movies.head(10)

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy
5,6,heat (1995),Action|Crime|Thriller
6,7,sabrina (1995),Comedy|Romance
7,8,tom and huck (1995),Adventure|Children's
8,9,sudden death (1995),Action
9,10,goldeneye (1995),Action|Adventure|Thriller


In [7]:
#타임 스탬프 칼럼 지우겠음
using_cols = ['user_id', 'movie_id', 'counts']
ratings = ratings[using_cols]
ratings.head(10)

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [8]:
ratings=pd.merge(ratings,movies)
ratings

,user_id,movie_id,counts,title,genre
0,1,1193,5,one flew over the cuckoo's nest (1975),Drama
1,2,1193,5,one flew over the cuckoo's nest (1975),Drama
2,12,1193,4,one flew over the cuckoo's nest (1975),Drama
3,15,1193,4,one flew over the cuckoo's nest (1975),Drama
4,17,1193,5,one flew over the cuckoo's nest (1975),Drama
...,...,...,...,...,...
836473,5851,3607,5,one little indian (1973),Comedy|Drama|Western
836474,5854,3026,4,slaughterhouse (1987),Horror
836475,5854,690,3,"promise, the (versprechen, das) (1994)",Romance
836476,5938,2909,4,"five wives, three secretaries and me (1998)",Documentary


## 분석해봅시다
- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순

In [9]:
#ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [10]:
#ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [11]:
#가장 인기 있는 영화 30개(인기순)
movie_count = ratings.groupby('title')['counts'].count()
movie_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [12]:
movie_count

title
$1,000,000 duck (1971)                         26
'burbs, the (1989)                            198
'night mother (1986)                           56
'til there was you (1997)                      27
...and justice for all (1979)                 185
                                             ... 
zachariah (1971)                                1
zed & two noughts, a (1985)                    24
zero effect (1998)                            262
zero kelvin (kjærlighetens kjøtere) (1995)      2
zeus and roxanne (1997)                        12
Name: counts, Length: 3628, dtype: int64

## 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [13]:
ratings

,user_id,movie_id,counts,title,genre
0,1,1193,5,one flew over the cuckoo's nest (1975),Drama
1,2,1193,5,one flew over the cuckoo's nest (1975),Drama
2,12,1193,4,one flew over the cuckoo's nest (1975),Drama
3,15,1193,4,one flew over the cuckoo's nest (1975),Drama
4,17,1193,5,one flew over the cuckoo's nest (1975),Drama
...,...,...,...,...,...
836473,5851,3607,5,one little indian (1973),Comedy|Drama|Western
836474,5854,3026,4,slaughterhouse (1987),Horror
836475,5854,690,3,"promise, the (versprechen, das) (1994)",Romance
836476,5938,2909,4,"five wives, three secretaries and me (1998)",Documentary


In [14]:
#nan값 다 지움 
ratings=ratings.dropna(axis=0)
ratings

,user_id,movie_id,counts,title,genre
0,1,1193,5,one flew over the cuckoo's nest (1975),Drama
1,2,1193,5,one flew over the cuckoo's nest (1975),Drama
2,12,1193,4,one flew over the cuckoo's nest (1975),Drama
3,15,1193,4,one flew over the cuckoo's nest (1975),Drama
4,17,1193,5,one flew over the cuckoo's nest (1975),Drama
...,...,...,...,...,...
836473,5851,3607,5,one little indian (1973),Comedy|Drama|Western
836474,5854,3026,4,slaughterhouse (1987),Horror
836475,5854,690,3,"promise, the (versprechen, das) (1994)",Romance
836476,5938,2909,4,"five wives, three secretaries and me (1998)",Documentary


In [15]:
my_favorite = ['requiem for a dream (2000)','two family house (2000)','tigerland (2000)','waiting to exhale (1995)','father of the bride part ii (1995)']
genre=['Drama','Drama','Drama','Comedy|Drama','Comedy']
mov_id=[3949,3951,3950,4,5]
my_movlist = pd.DataFrame({'user_id': ['soon']*5,'movie_id':mov_id ,'title': my_favorite,'genre':genre,'counts':[3]*5})

if not ratings.isin({'user_id':['soon']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_movlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,title,genre
836473,5851,3607,5,one little indian (1973),Comedy|Drama|Western
836474,5854,3026,4,slaughterhouse (1987),Horror
836475,5854,690,3,"promise, the (versprechen, das) (1994)",Romance
836476,5938,2909,4,"five wives, three secretaries and me (1998)",Documentary
836477,5948,1360,5,identification of a woman (identificazione di ...,Drama
0,soon,3949,3,requiem for a dream (2000),Drama
1,soon,3951,3,two family house (2000),Drama
2,soon,3950,3,tigerland (2000),Drama
3,soon,4,3,waiting to exhale (1995),Comedy|Drama
4,soon,5,3,father of the bride part ii (1995),Comedy


In [16]:
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [17]:
print(user_to_idx['soon'])

6039


In [18]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('mociet column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

ratings

user_id column indexing OK!!
mociet column indexing OK!!


,user_id,movie_id,counts,title,genre
0,0,1193,5,0,Drama
1,1,1193,5,0,Drama
2,2,1193,4,0,Drama
3,3,1193,4,0,Drama
4,4,1193,5,0,Drama
...,...,...,...,...,...
0,6039,3949,3,1318,Drama
1,6039,3951,3,2498,Drama
2,6039,3950,3,2398,Drama
3,6039,4,3,397,Comedy|Drama


## 4) CSR matrix를 직접 만들어 봅시다.

In [19]:
num_user = ratings['user_id'].nunique()
num_user

6040

In [20]:
num_movie = ratings['title'].nunique()
num_movie

3628

In [21]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.title)), shape= (num_user, num_movie))

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [22]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [33]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model1= AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=50, dtype=np.float32)

In [34]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [35]:
# 모델 훈련
als_model1.fit(csr_data_transpose)

  0%|          | 0/50 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

#### 5개 추가로 넣은 영화중 하나인 tigerland로 실험

In [36]:
soon,tigerland = user_to_idx['soon'], movie_to_idx['tigerland (2000)']
soon_vector, tigerland_vector = als_model1.user_factors[soon], als_model1.item_factors[tigerland]


In [37]:
soon_vector

array([ 0.04097557, -0.10422312, -0.17610793, -0.19347431,  0.29293337,
        0.09011723,  0.18903369, -0.04003558,  0.14696571,  0.03467905,
        0.10111357,  0.01686911, -0.32734078,  0.1900918 , -0.09185813,
        0.08537172, -0.05421869,  0.06121325,  0.11651979, -0.1544339 ,
       -0.08989438,  0.16226453, -0.06750368, -0.06183898, -0.02848781,
       -0.13323328,  0.07562458,  0.04987546, -0.05214698,  0.21303214,
        0.07483377, -0.14021508, -0.07392525,  0.23905426, -0.16929254,
        0.27070385,  0.07289355, -0.0178685 ,  0.11933191, -0.09702702,
        0.09976662,  0.05002536, -0.11771712,  0.07815167,  0.2839128 ,
       -0.08470601,  0.25745586, -0.10580559,  0.1667664 , -0.06814303,
        0.20499814,  0.04311161, -0.01721488, -0.07860804, -0.21727696,
        0.22605054, -0.19993906,  0.21354601,  0.25826618, -0.13258487,
        0.0280307 , -0.03884595, -0.12586759, -0.11215744, -0.12139384,
        0.13612482,  0.04562961, -0.13148654,  0.17375153, -0.12

In [38]:
tigerland_vector

array([ 1.25604160e-02,  2.59922491e-03,  1.09720300e-03,  4.65690624e-03,
        1.94657445e-02,  8.36833846e-03,  9.18601640e-03,  2.51018088e-02,
        9.73908976e-03, -4.84873448e-03,  7.51865748e-03,  4.76274593e-03,
        4.99077607e-04,  3.72714712e-03, -4.07281425e-03,  2.99970270e-03,
       -5.12787839e-03,  1.54936761e-02,  8.68950505e-03, -3.31092207e-03,
        8.10554065e-03,  9.04307235e-03,  9.06218588e-03,  4.82804608e-03,
        9.42173880e-03,  5.03586186e-03,  1.14233932e-02,  1.36373779e-02,
        8.87936912e-03,  1.60714053e-02,  2.68345978e-03,  7.41332211e-03,
        4.44053812e-03,  5.87597676e-03,  3.19854892e-03,  1.55467438e-02,
        8.02647136e-03,  1.54990572e-04,  9.78856068e-03, -4.98515554e-04,
        1.17037222e-02,  7.74328131e-03,  6.94237184e-03,  9.09936894e-03,
        4.98855906e-03,  1.10175889e-02,  7.72963604e-03, -6.78065990e-04,
        1.37291374e-02,  1.94617026e-02, -4.27446328e-03,  1.07813179e-02,
        2.80104019e-03,  

In [39]:
np.dot(soon_vector, tigerland_vector)

0.11658661

#### factors 와 iteration 수를 늘리니 0.07에서 0.1까지 증가하였다

In [ ]:
## contender, the (2000), 드라마 스릴러 

In [41]:
rr = movie_to_idx['contender, the (2000)']
rr_vector = als_model1.item_factors[rr]
np.dot(soon_vector, rr_vector)

0.20007332

#### 좋아하는 영화 모두 드라마 장르가 필수적으로 들어있어서 ,드라마 장르가 포함된 드라마 스릴러 영화 장르를 하나 넣어봤는데 0.2정도 관계도를 보인다.. 

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [44]:
favorite_movie = 'toy story (1995)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model1.similar_items(movie_id, N=15)
similar_movie

[(40, 1.0),
 (50, 0.5653399),
 (33, 0.325613),
 (4, 0.32449278),
 (322, 0.31235576),
 (330, 0.27171442),
 (255, 0.24997225),
 (110, 0.24591392),
 (10, 0.23548427),
 (2938, 0.20302095),
 (126, 0.19583596),
 (288, 0.19401854),
 (3589, 0.19400373),
 (160, 0.19222268),
 (1736, 0.19179791)]

In [45]:
#, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['toy story (1995)',
 'toy story 2 (1999)',
 'aladdin (1992)',
 "bug's life, a (1998)",
 'babe (1995)',
 'lion king, the (1994)',
 "there's something about mary (1998)",
 'groundhog day (1993)',
 'beauty and the beast (1991)',
 'nobody loves me (keiner liebt mich) (1994)',
 'shakespeare in love (1998)',
 'austin powers: international man of mystery (1997)',
 'soft toilet seats (1999)',
 'forrest gump (1994)',
 'small soldiers (1998)']

#### !!!! 오 토이 스트리로 추천을 받아보니 토이 스토리 시리즈 추천과 버그 라이프, 라이온킹 등 애니메이션 위주로 추천해주고 있다

In [47]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model1.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie


#### 스타워즈나 매트릭스로 추천을 한번 더 받아보겠당

In [48]:
get_similar_movie('star wars: episode iv - a new hope (1977)')

['star wars: episode iv - a new hope (1977)',
 'star wars: episode v - the empire strikes back (1980)',
 'raiders of the lost ark (1981)',
 'star wars: episode vi - return of the jedi (1983)',
 'alien (1979)',
 'star wars: episode i - the phantom menace (1999)',
 'simon sez (1999)',
 'loser (1991)',
 'match, the (1999)',
 'bushwhacked (1995)']

In [49]:
get_similar_movie('matrix, the (1999)')

['matrix, the (1999)',
 'terminator 2: judgment day (1991)',
 'total recall (1990)',
 'terminator, the (1984)',
 'fugitive, the (1993)',
 'fifth element, the (1997)',
 'face/off (1997)',
 'running free (2000)',
 'star wars: episode v - the empire strikes back (1980)',
 'surviving picasso (1996)']

추천 시스템이 잘 작동하고 있다. 같은 판타지 계열인 것과 시리즈 물의 경우 후속 시리즈 물들을 추천해주고 있다

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [50]:
user = user_to_idx['soon']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model1.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(1099, 0.2000733),
 (1853, 0.18330666),
 (755, 0.17274795),
 (1852, 0.15515065),
 (1814, 0.12714317),
 (2537, 0.12584895),
 (461, 0.11412127),
 (859, 0.10772695),
 (1117, 0.09920907),
 (904, 0.09691189),
 (254, 0.08981713),
 (2027, 0.08960904),
 (1887, 0.089246005),
 (1470, 0.088773765),
 (878, 0.08842312),
 (788, 0.08746851),
 (1857, 0.084600545),
 (41, 0.08167934),
 (1808, 0.081001714),
 (2243, 0.08008744)]

In [53]:
[idx_to_movie[i[0]] for i in artist_recommended]

['contender, the (2000)',
 'best in show (2000)',
 'almost famous (2000)',
 'dancer in the dark (2000)',
 'nurse betty (2000)',
 'girlfight (2000)',
 'meet the parents (2000)',
 'cell, the (2000)',
 'virgin suicides, the (1999)',
 'clockwork orange, a (1971)',
 'pi (1998)',
 'croupier (1998)',
 'wonder boys (2000)',
 'tao of steve, the (2000)',
 'strange days (1995)',
 'bringing out the dead (1999)',
 'bring it on (2000)',
 'rain man (1988)',
 'chuck & buck (2000)',
 'yards, the (1999)']

## 회고 

바로 위에서 추천 받은 영화들 장르 대부분이 드라마이다. 
추가로 넣은 영화들 모두 드라마 장르가 공통되게 넣어서 가장 첫번째 영화로 contender 영화가 드라마 스릴러 장르로 드라마 계열 영화가 출력되는 것을 볼 수 있다.



추천 시스템에서는 말했듯이 아주 추천이 잘된다.

하지만 내적을 계산하여 직접 값의 수치를 보는 것은 잘 이행되지 않았다. 앞서 말했듯이 contender 영화로 내적 값을 수행하였을 떄 0.2 밖에 나오지 않았다. 같은 조건으로 추천 받았을 떄 contender 영화가 추천 시스템으로 바로 상위권에 뜰 정도라면 0.8 정도의 내적 값이 나올줄 알았는데.. 0.2 라니.. 이것도 높은건가? 수치 값은 사실 잘 와닿지 않는다